pandasで  
groupbyして合計や平均を出すのを、条件を満たす一部に対して実行したい。  
……あんまり参考になる質問がないんだよねー。

https://stackoverflow.com/questions/17266129/python-pandas-conditional-sum-with-groupby  
33 votes, PV 44000。  
https://stackoverflow.com/questions/45752601/how-to-do-a-conditional-count-after-groupby-on-a-pandas-dataframe  
34 votes、PV 47000。


小規模なサンプルデータだと「どういうときに使うのかが分かりにくいので、前処理大全のデータを借ります。

In [1]:
import pandas as pd

In [2]:
reserve_tb = pd.read_csv('https://raw.githubusercontent.com/ghmagazine/awesomebook/master/data/reserve.csv')
hotel_tb = pd.read_csv('https://raw.githubusercontent.com/ghmagazine/awesomebook/master/data/hotel.csv')

In [3]:
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


## Q1. 各ホテルについて、予約の合計金額はいくらか? 

In [4]:
reserve_tb.groupby('hotel_id').sum()

,people_num,total_price
hotel_id,,
h_1,26,1122300
h_10,8,128800
h_100,54,552000
h_101,41,1288000
h_102,36,426000
...,...,...
h_95,34,3585600
h_96,34,429200
h_97,35,1337600


In [5]:
# 上記で間違ってはいないが、ホテルのIDが順番になっていないので見づらい。
# 順序を変更するにはreindex関数を使う

reserve_tb.groupby('hotel_id').sum().reindex(hotel_tb['hotel_id'])

,people_num,total_price
hotel_id,,
h_1,26,1122300
h_2,24,1188000
h_3,44,4130000
h_4,32,306800
h_5,27,661500
...,...,...
h_296,38,1083600
h_297,15,900000
h_298,25,392000


## Q2. （一部の予約について、ホテルごとに）宿泊人数が3人である予約だけに対して、各ホテルについて、予約の合計金額はいくらか?

In [6]:
reserve_tb.query('people_num == 3').groupby('hotel_id').sum()

,people_num,total_price
hotel_id,,
h_1,9,391500
h_10,3,50400
h_100,15,158400
h_101,15,462000
h_102,18,198000
...,...,...
h_94,15,423900
h_95,3,259200
h_96,15,199800


## Q3. Q2の結果の金額をhotel_tbの新たな列として追加することはできるか?

In [7]:
hotel_tb.head()

,hotel_id,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,h_1,26100,D,D-2,43.064569,141.511397,True
1,h_2,26400,A,A-1,35.715320,139.939446,True
2,h_3,41300,E,E-4,35.281572,136.988565,False
3,h_4,5200,C,C-3,38.431293,140.795615,False
4,h_5,13500,G,G-3,33.597291,130.533872,True


In [8]:
# これはダメ
# hotel_tbのindexが0, 1, 2, ……だが、右辺はgroupbyした結果だからindexはh_1, h_2, ……であり、indexが一致しない
hotel_tb['3_people_total_price'] = reserve_tb.query('people_num == 3').groupby('hotel_id')['total_price'].sum()

In [9]:
hotel_tb.head()

,hotel_id,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business,3_people_total_price
0,h_1,26100,D,D-2,43.064569,141.511397,True,NaN
1,h_2,26400,A,A-1,35.715320,139.939446,True,NaN
2,h_3,41300,E,E-4,35.281572,136.988565,False,NaN
3,h_4,5200,C,C-3,38.431293,140.795615,False,NaN
4,h_5,13500,G,G-3,33.597291,130.533872,True,NaN


In [10]:
hotel_tb.merge(reserve_tb.query('people_num == 3').groupby('hotel_id')['total_price'].sum(), left_on='hotel_id', right_on=index)

NameError: name 'index' is not defined

In [11]:
hotel_tb.merge(reserve_tb.query('people_num == 3').groupby('hotel_id')['total_price'].sum(), left_on='hotel_id', right_index=True)

# todo total_priceを別の列名に変更する

,hotel_id,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business,3_people_total_price,total_price
0,h_1,26100,D,D-2,43.064569,141.511397,True,NaN,391500
2,h_3,41300,E,E-4,35.281572,136.988565,False,NaN,991200
3,h_4,5200,C,C-3,38.431293,140.795615,False,NaN,31200
4,h_5,13500,G,G-3,33.597291,130.533872,True,NaN,486000
5,h_6,49500,A,A-3,35.912764,139.731281,True,NaN,1336500
...,...,...,...,...,...,...,...,...,...
295,h_296,17200,F,F-4,34.638426,132.564713,True,NaN,154800
296,h_297,30000,A,A-3,35.815512,139.940259,False,NaN,630000
297,h_298,9800,A,A-1,35.717254,139.738926,False,NaN,147000
298,h_299,24200,C,C-2,38.335725,140.797714,False,NaN,798600


こういう分析をするときに、パターンが増えて
「何らかの条件を満たす予約だけに対して、各ホテルについて、予約の合計金額はいくらか?」を見たいときに  
一々mergeするのは面倒じゃない?  
`hotel_tb['3_people_total_price']= ...` で書けたら嬉しくない?  
と思うけど。  
どうやるのが良いのだろう。